# Hotel Reservation - Data Prep

In this notebook we focus on the data loading, basic exploring, and preparation for the hotel reservation dataset from kaggle.


## Import and install python libraries

In [68]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import imblearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

np.random.seed(1)

### Loading the data

In [69]:
hotel_res = pd.read_csv("C:/Users/simra/Downloads/Hotel Reservations.csv")
hotel_res.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


### Basic evaluation
We have a number of input variables and one target variable. For this analysis, the target variable is price.

First, our initial exploration of the data should answer the following questions:
1. How many rows and columns
2. How much of a problem do we have with na's?
3. What types of data are there?
4. What types of data are stored in columns
    1. identify which variables are numeric and may need to be standardized later
    2. identify which variables are categorical and may need to be transformed using and encoders such as one-hot-encoder.
5. Identify errors in the data - this is a common problem with categorical vars where the category is mispelled or spelled differently in some instances.

In [70]:
hotel_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [71]:
hotel_res.isna().sum()  #checking for null value columns if any

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

In [72]:
category_var_list = list(hotel_res.select_dtypes(include='object').columns) #catagorical variables
category_var_list

['Booking_ID',
 'type_of_meal_plan',
 'room_type_reserved',
 'market_segment_type',
 'booking_status']

In [73]:
#checking for misspelled category values

[print(f"Category: {cat} Values: {hotel_res[cat].unique()}") for cat in category_var_list] 

Category: Booking_ID Values: ['INN00001' 'INN00002' 'INN00003' ... 'INN36273' 'INN36274' 'INN36275']
Category: type_of_meal_plan Values: ['Meal Plan 1' 'Not Selected' 'Meal Plan 2' 'Meal Plan 3']
Category: room_type_reserved Values: ['Room_Type 1' 'Room_Type 4' 'Room_Type 2' 'Room_Type 6' 'Room_Type 5'
 'Room_Type 7' 'Room_Type 3']
Category: market_segment_type Values: ['Offline' 'Online' 'Corporate' 'Aviation' 'Complementary']
Category: booking_status Values: ['Not_Canceled' 'Canceled']


[None, None, None, None, None]

### Summary the findings from our initial evaluation of the data

* We have 4 categorical variables
* We have no variables that have missing values
* There doesn't seem to be a problem with the catogorical class names.

## Process the data

* Conduct any data prepartion that should be done *BEFORE* the data split.
* Split the data.
* Conduct any data preparation that should be done *AFTER* the data split.

### Data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. We can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

In [74]:
#dropping the id column as it is a unique identifier which can't be used for model training or testing
hotel_res.drop(['Booking_ID'], axis=1, inplace = True) 

#### Encode our categorical variables
Categorical variables usually have strings for their values. Many machine learning algorithms do not support string values for the input variables. Therefore, we need to replace these string values with numbers. This process is called categorical variable encoding.

We have three main approaches to encoding variables
* One-Hot-Encoding
* Dummy Encoding
* Label Encoding

In this assignment; we will dummy encode booking_status using dummy encoding and type_of_meal_plan, room_type_reserved, market_segment_type using one-hot encoding.

In [75]:
hotel_res = hotel_res.join(pd.get_dummies(hotel_res['type_of_meal_plan'], prefix='type_of_mp', drop_first=False))
hotel_res.drop('type_of_meal_plan', axis=1, inplace = True)

In [76]:
hotel_res = hotel_res.join(pd.get_dummies(hotel_res['room_type_reserved'], prefix='reserved', drop_first=False))
hotel_res.drop('room_type_reserved', axis=1, inplace = True)

In [77]:
hotel_res = hotel_res.join(pd.get_dummies(hotel_res['market_segment_type'], prefix='market_segment_type', drop_first=False))
hotel_res.drop('market_segment_type', axis=1, inplace = True)

In [78]:
hotel_res = hotel_res.join(pd.get_dummies(hotel_res['booking_status'], prefix='booking_status', drop_first=True))
hotel_res.drop('booking_status', axis=1, inplace = True)

In [79]:
hotel_res.head(3)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,reserved_Room_Type 4,reserved_Room_Type 5,reserved_Room_Type 6,reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Not_Canceled
0,2,0,1,2,0,224,2017,10,2,0,...,0,0,0,0,0,0,0,1,0,1
1,2,0,2,3,0,5,2018,11,6,0,...,0,0,0,0,0,0,0,0,1,1
2,1,0,2,1,0,1,2018,2,28,0,...,0,0,0,0,0,0,0,0,1,0


### Split the data into validation and training set

In [80]:

train_df, test_df = train_test_split(hotel_res, test_size=0.3)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'booking_status_Not_Canceled'
predictors = list(hotel_res.columns)
predictors.remove(target)

### Data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numeric variables

In [81]:
hotel_res.select_dtypes(include='int64').columns

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests'],
      dtype='object')

In [82]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests','avg_price_per_room']    

#in the above list from above cell the float column was not there, that is also added 'avg_price_per_room'        
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


### Handling the imbalanced dataset

In [83]:
hotel_res['booking_status_Not_Canceled'].value_counts() #to check imbalance in the data

1    24390
0    11885
Name: booking_status_Not_Canceled, dtype: int64

In [84]:
train_X = train_df[predictors]
train_y = train_df[target] # train_target is now a series object
test_X = test_df[predictors]
test_y = test_df[target] # validation_target is now a series object

In [85]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from collections import Counter

undersample = RandomUnderSampler(sampling_strategy='majority')

train_X, train_y = undersample.fit_resample(train_X, train_y)

print(Counter(train_y))

Counter({0: 8363, 1: 8363})


### Saving the preprocessed files

In [86]:
train_df.to_csv('hotel_res_train_df.csv', index=False)
train_X.to_csv('hotel_res_train_X.csv', index=False)
train_y.to_csv('hotel_res_train_y.csv', index=False)
test_df.to_csv('hotel_res_test_df.csv', index=False)
test_X.to_csv('hotel_res_test_X.csv', index=False)
test_y.to_csv('hotel_res_test_y.csv', index=False)